In [1]:
import torch
import numpy as np
import torch.optim as opt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
with open('reviews.txt', 'r') as f:
  reviews = f.read()

with open('labels.txt', 'r') as f:
  labels = f.read()

In [3]:
print(reviews[:100], end = '\n')
print(labels[:100])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
n


In [4]:
from string import punctuation

print(punctuation)

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')

all_text = ' '.join(reviews_split)

words = all_text.split()

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [7]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key = counts.get, reverse=True)
vocab_to_int = {word:ii for ii, word in enumerate(vocab, 1)}

review_ints = []
for review in reviews_split:
  review_ints.append([vocab_to_int[word] for word in review.split()])

In [8]:
print("Number of Unique words {}".format(len(vocab)) )
print("First review after numerical encoding")
print(review_ints[0])

Number of Unique words 74072
First review after numerical encoding
[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]


In [9]:
labels_split = labels.split('\n')

targets = [1 if label == 'positive' else 0 for label in labels_split]

In [10]:
encoded_labels = np.array(targets)

In [12]:
review_len = Counter([len(x) for x in review_ints])

print(f"No of zero length reviews {review_len[0]}")
print(f"Longest review is of size {max(review_len)}")

No of zero length reviews 1
Longest review is of size 2514


In [13]:
print("No of reviews before removing zero length reviews {}".format(len(review_ints)))

non_zero_idx = [ii for ii, review in enumerate(review_ints) if len(review)!=0]

review_ints = [review_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print("No of reviews after removing zero length reviews {}".format(len(review_ints)))

No of reviews before removing zero length reviews 25001
No of reviews after removing zero length reviews 25000


In [15]:
def pad_seqeunces(arr, seq_len):
  features = np.zeros((len(arr), seq_len), dtype=int)

  for i, row in enumerate(arr):
    features[i, -len(row):] = np.array(row)[:seq_len]
  
  return features

In [16]:
seq_length = 200

features = pad_seqeunces(review_ints, seq_length)

assert len(features)==len(review_ints), 'Features should have as many rows as review_ints'
assert len(features[0])==seq_length, 'Features should have seq_length no. of columns'

print(features[:30, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [20]:
split_frac = 0.8

split_idx = int(len(features)*split_frac)
train_x, remain_x = features[:split_idx], features[split_idx:]
train_y, remain_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

split_idx = int(0.5*len(remain_x))
eval_x, test_x = remain_x[:split_idx], remain_x[split_idx:]
eval_y, test_y = remain_y[:split_idx], remain_y[split_idx:]

print(train_x.shape)
print(eval_x.shape)
print(test_x.shape)

(20000, 200)
(2500, 200)
(2500, 200)


In [21]:
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
eval_data = TensorDataset(torch.from_numpy(eval_x), torch.from_numpy(eval_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

trainLoader = DataLoader(train_data, shuffle = True, batch_size = batch_size)
testLoader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
evalLoader = DataLoader(eval_data, shuffle=True, batch_size=batch_size)

In [22]:
dataiter = iter(trainLoader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) 
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) 
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   10,   431,     5,  ...,   171,  1623,   461],
        [  772,     1,  7054,  ...,     1,   330,   112],
        [ 1052,     3,   318,  ...,    33,   967,   129],
        ...,
        [  786,    16,    60,  ..., 13900,    38,  2235],
        [  596,   127,     6,  ...,   137,    70,   847],
        [    0,     0,     0,  ...,    92,   203,    50]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
        1, 0])


In [23]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("Training on GPU")
else:
  print("Training on CPU")

Training on GPU


In [40]:
class SentimentRNN(nn.Module):
  def __init__(self, token_size, ouput_size, embedding_dim, hidden_dim, n_layers, drop_prob = 0.5):
    super(SentimentRNN, self).__init__()

    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.output_size = output_size

    self.embedding = nn.Embedding(token_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,dropout = drop_prob, batch_first = True)

    self.dropout = nn.Dropout(0.3)

    self.fc = nn.Linear(hidden_dim, output_size)

    self.sig = nn.Sigmoid()

  def forward(self, x, hidden):
    x = x.long()

    batch_size = x.size(0)

    embeds = self.embedding(x)

    lstm_outs, hidden = self.lstm(embeds, hidden)

    lstm_outs = lstm_outs[:, -1, :]

    out = self.fc(lstm_outs)
    out = self.dropout(out)

    sigs = self.sig(out)

    return sigs, hidden

  def init_hidden(self, batch_size):
    weights = next(net.parameters()).data

    if train_on_gpu:
      hidden = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
    
    return hidden

In [41]:
token_size = len(vocab_to_int) + 1
output_size = 1
hidden_dim = 256
embedding_size = 400
n_layers = 2

net = SentimentRNN(token_size, output_size, embedding_size, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [42]:
lr = 0.001

criterion = nn.BCELoss()

optimizer = opt.Adam(net.parameters(), lr = lr)

In [44]:
epochs = 4 

counter = 0
print_every = 100
clip=5 


if(train_on_gpu):
    net.cuda()

net.train()

for e in range(epochs):

    h = net.init_hidden(batch_size)


    for inputs, labels in trainLoader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        h = tuple([each.data for each in h])


        net.zero_grad()


        output, h = net(inputs, h)


        loss = criterion(output.squeeze(), labels.float())
        loss.backward()

        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()


        if counter % print_every == 0:

            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in evalLoader:


                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))


Epoch: 1/4... Step: 100... Loss: 0.635163... Val Loss: 0.677931
Epoch: 1/4... Step: 200... Loss: 0.605607... Val Loss: 0.666471
Epoch: 1/4... Step: 300... Loss: 0.591498... Val Loss: 0.574938
Epoch: 1/4... Step: 400... Loss: 0.605294... Val Loss: 0.581496
Epoch: 2/4... Step: 500... Loss: 0.465846... Val Loss: 0.525107
Epoch: 2/4... Step: 600... Loss: 0.596905... Val Loss: 0.551626
Epoch: 2/4... Step: 700... Loss: 0.518762... Val Loss: 0.532923
Epoch: 2/4... Step: 800... Loss: 0.413665... Val Loss: 0.498129
Epoch: 3/4... Step: 900... Loss: 0.567285... Val Loss: 0.454341
Epoch: 3/4... Step: 1000... Loss: 0.513801... Val Loss: 0.458865
Epoch: 3/4... Step: 1100... Loss: 0.602542... Val Loss: 0.448647
Epoch: 3/4... Step: 1200... Loss: 0.415384... Val Loss: 0.450705
Epoch: 4/4... Step: 1300... Loss: 0.393313... Val Loss: 0.448229
Epoch: 4/4... Step: 1400... Loss: 0.350736... Val Loss: 0.443219
Epoch: 4/4... Step: 1500... Loss: 0.395996... Val Loss: 0.449715
Epoch: 4/4... Step: 1600... Loss: 

In [47]:
test_losses = [] 
num_correct = 0


h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in testLoader:


    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    

    output, h = net(inputs, h)
    

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    

    pred = torch.round(output.squeeze())
    
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)



print("Test loss: {:.3f}".format(np.mean(test_losses)))


test_acc = num_correct/len(testLoader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.453
Test accuracy: 0.796


In [48]:
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [54]:
def tokenize_review(test_review):
    test_review = test_review.lower()
    
    test_text = ''.join([c for c in test_review if c not in punctuation])

    
    test_words = test_text.split()

    
    test_ints = []
    test_ints.append([vocab_to_int.get(word, 0) for word in test_words])

    return test_ints

In [55]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    
    test_ints = tokenize_review(test_review)
    
    
    seq_length=sequence_length
    features = pad_seqeunces(test_ints, seq_length)
    
    
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    
    output, h = net(feature_tensor, h)
    
    
    pred = torch.round(output.squeeze()) 
    
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [56]:
predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.027085
Negative review detected.


In [58]:
predict(net, "Movie was not good", seq_length)

Prediction value, pre-rounding: 0.463442
Negative review detected.


In [60]:
predict(net, "Loved it!", seq_length)

Prediction value, pre-rounding: 0.901594
Positive review detected!
